In [12]:
import numpy as np
import networkx as nx
import itertools

In [34]:
#load the binary tree adjacency matrix.
#graph = np.loadtxt("adjacency_tree.txt", int)
#graph = nx.to_numpy_matrix(nx.petersen_graph(), dtype=np.int)
graph = nx.to_numpy_matrix(nx.path_graph(5), dtype=int).astype(int)
print(graph)

[[0 1 0 0 0]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [0 0 1 0 1]
 [0 0 0 1 0]]


In [14]:
# Order - simply count the number of rows
order = graph.shape[1]
print(order)

7


In [15]:
#Size - count all the entries in the matrix, divide by 2 (since each edge is listed twice)
size = np.sum(graph)/2
print(size)

6.0


In [16]:
# Max degree, min degree, and degree sequence
degseq = sorted(np.sum(graph, axis=1), reverse=True)
maxdeg = max(degseq)
mindeg = min(degseq)

print(degseq)
print(maxdeg)
print(mindeg)

[3, 3, 2, 1, 1, 1, 1]
3
1


In [17]:
# Eccentricity, radius and diameter
def ecc(row):
    found = [row]
    depth = 0
    # Repeat until we found all verticies
    while set(range(graph.shape[0])) > set(found):
        foundcpy = found[:]
        depth = depth + 1
        for x in found:
            for y in range(graph.shape[0]):
                if graph[x,y] == 1 and y not in foundcpy:
                    foundcpy.append(y)
        found = foundcpy
    return depth

ecclist = {}
for x in range(graph.shape[0]):
    # print("Checking %d , %d" % (x,y))
    ecclist[x] = ecc(x)

ecclist = list(ecclist.values())
radius = min(ecclist)
diameter = max(ecclist)

print(ecclist)
print(diameter)
print(radius)

[4, 4, 3, 2, 3, 4, 4]
4
2


In [18]:
# Brute force algo for the chromatic number.
# Start with 2 colors
cnum = 1
valid = False
while not valid:
    # print("Trying %d colors" % cnum)
    for coloring in itertools.product(range(cnum), repeat=graph.shape[0]):
        # this gives us something like (0, 0, 1, 0, 1)
        # each number is a color
        # check all adjacencies of all edges, if color matches, break and check next coloring
        if valid == False: # if valid still true, we found a coloring, can stop checking
            valid = True
            for x in range(graph.shape[0] - 1):
                if valid: # Saves some time when checking
                    for y in range(graph.shape[0] - 1):
                        # check adjacency
                        if x!=y and graph[x,y] == 1:
                            # check 
                            valid = valid and (coloring[x] != coloring[y])
    if not valid:
        cnum = cnum + 1
print("Chromatic number is %d" % cnum)

Chromatic number is 2


In [19]:
# Neighbors helper function, used in a few things.
def neighbors(vertex):
    nset = []
    for y in range(graph.shape[0]):
        if graph[vertex,y] == 1:
            nset.append(y)
    return nset

# Powerset helper function, used for brute force algorithms.
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

# Independence number brute force algo.
indep = 0
for vset in powerset(range(graph.shape[0])):
    valid = True
    for node in vset:
        if any(i in vset for i in neighbors(node)): # Check if any neighbors are in our set (i.e. not independent)
            valid = False
    if valid:
        indep = len(vset)
print(indep)

5


In [20]:
# Clique number.
# Python implementation of the Bron–Kerbosch algorithm.
# We don't need to return the clique with the number, just the number itself.

clique_number = 0

def BronKerbosch(R, P, X):
    global clique_number
    if len(P) == 0 and len(X) == 0:
        if len(R) > clique_number:
            clique_number = len(R)
    for v in P[:]:
        R_v = R + [v]
        P_v = list(set(P).intersection(neighbors(v)))
        X_v = list(set(X).intersection(neighbors(v)))
        BronKerbosch(R_v, P_v, X_v)
        P.remove(v)
        X.append(v)
BronKerbosch([], list(range(graph.shape[0] - 1)), [])

print(clique_number)

2


In [21]:
# Domination number.
# Brute force check of all the vertex sets.
def total_domination_number():
    for vset in powerset(range(graph.shape[0])):
        nset = np.zeros(graph.shape[0], int)
        for v in vset:
            np.bitwise_or(nset, graph[v], nset) # bonus points for using bitwise or
        if sum(nset) == graph.shape[0]:
            return len(vset)

def domination_number():
    for vset in powerset(range(graph.shape[0])):
        nset = np.zeros(graph.shape[0], int)
        for v in vset:
            nset[v] = 1  # Only difference from total domination - start with vertexes
            np.bitwise_or(nset, graph[v], nset)
        if sum(nset) == graph.shape[0]:
            return len(vset)      

print(total_domination_number())
print(domination_number())

3
2


In [35]:
# Brute force zero forcing number algorithm
# Check all neighbors of each vertex and see if they are in the set.
# If there is one and only one neighbor, add it to the set, and repeat until we can't add any new neighbors
def coloring_rec(vset):
    vset_c = vset[:]
    for vertex in vset:
        ncn = [] 
        for neighbor in neighbors(vertex):
            if neighbor not in vset_c:
                ncn.append(neighbor)
        if len(ncn) == 1: # Append the nonconnected neighbor to vset.
            vset_c.append(ncn[0])
        # did we add new nodes?  if so, run coloring again
    if vset_c != vset:
        vset_r = vset_c[:]
        return coloring_rec(vset_r)
    else:
        return vset

zero_forcing = -1
    
for vset in powerset(range(graph.shape[0])):
    vcopy = coloring_rec(list(vset))
    if list(range(graph.shape[0])) == sorted(list(vcopy)):
        # found one - return it
        print(vset)
        zero_forcing = len(vset)
        break

print(zero_forcing)

()
[]
(0,)
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
1


In [23]:
neighbors(3)

[2, 4]

In [24]:
print(list(range(7)))

[0, 1, 2, 3, 4, 5, 6]
